# LAV BEV Training on Colab

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/MichaelBosello/carla-colab/blob/master/carla-simulator.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
</table>

---
---

## Introduction
[Carla](https://github.com/carla-simulator/carla) is an open-source simulator for autonomous driving research.

Since it is hardware demanding, it could be useful for students to run it on colab.

---

## Acknowledgements

Thanks to *Tomohiro*  for developing the code that allows remote access to colab.
Check out his [page](https://internet-of-tomohiro.netlify.app/google_colab/vnc.en.html) and [GitHub](https://github.com/demotomohiro/remocolab).



## Installation 

4. Wait the end of the process
  *



In [ ]:
# !nvidia-smi # list your current GPU


## Download Jekyll's LAV

Clone myown LAV from(https://github.com/jekyll88/lav)


In [1]:
! git clone https://github.com/jekyll88/lav.git


Cloning into 'lav'...
remote: Enumerating objects: 1972, done.
remote: Counting objects: 100% (1972/1972), done.
remote: Compressing objects: 100% (1838/1838), done.
remote: Total 1972 (delta 128), reused 1964 (delta 123), pack-reused 0
Receiving objects: 100% (1972/1972), 34.55 MiB | 26.76 MiB/s, done.
Resolving deltas: 100% (128/128), done.


## Install requirements


In [ ]:
!pip3 install -r lav/requirements.txt

## Install necessary packages

In [ ]:
#!wget https://github.com/jekyll88/lav/blob/main/carla/dist/carla-0.9.13-cp37-cp37m-manylinux_2_27_x86_64.whl

!pip3 install lav/carla/dist/carla-0.9.13-cp37-cp37m-manylinux_2_27_x86_64.whl
!pip3 install -r lav/environment.yaml
import sys
sys.path.append('/content/lav')

## Preparing Data


In [6]:
import tqdm
import torch
from lav.lav_privileged import LAV
from lav.utils.datasets import get_data_loader
from lav.utils.logger import Logger


## Training

In [7]:
#from google.colab import drive
#drive.mount('/content/drive')

dmd = LAV(args)
data_loader = get_data_loader('bev', args)
logger = Logger('lav_bev', args)
save_dir = logger.save_dir

torch.manual_seed(args.seed)

# logger.watch_model(dmd.uniplanner)

global_it = 0
for epoch in range(args.num_epoch):
    for data in tqdm.tqdm(data_loader, desc=f'Epoch {epoch}'):

        opt_info = dmd.train_bev(*data)

        if global_it % args.num_per_log == 0:
            logger.log_bev_info(global_it, opt_info)

        global_it += 1

    dmd.bev_scheduler.step()

    if (epoch+1) % args.num_per_save == 0:
        bev_path = f'{save_dir}/bev_{epoch+1}.th'
        torch.save(dmd.state_dict('bev'), bev_path)
        print (f'save to {bev_path}')

        logger.save([bev_path])

NameError: ignored